# Setup
This section contains all necessary setup such as libraries and model creation.

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

from src.data.data_helper import get_raw_data_as_dataframe

In [2]:
raw_data = get_raw_data_as_dataframe()
raw_data.head()

,time,measurement,label,source
0,0.0000,1.610419,0,C:\Users\erik\IES_codebase\EMG_Project\CDT406-...
1,0.0002,1.610419,0,C:\Users\erik\IES_codebase\EMG_Project\CDT406-...
2,0.0004,1.610419,0,C:\Users\erik\IES_codebase\EMG_Project\CDT406-...
3,0.0006,1.610419,0,C:\Users\erik\IES_codebase\EMG_Project\CDT406-...
4,0.0008,1.610419,0,C:\Users\erik\IES_codebase\EMG_Project\CDT406-...


## Pre-processor Calibration
Due to the fact that all data needs to be pre-processed using the same bandpass filter and normalization the pre-processor needs to be calibrated. This is done by calculating the coefficients of the bandpass filter and the normalization parameters.

These parameters will then be passed to the selected model class.

In [3]:
from src.models.preprocessing.preprocessor import SignalPreprocessor

pre_processor = SignalPreprocessor(
    low_freq=20.0,
    high_freq=500.0,
    fs=5000.0,
    order=4
)

In [4]:
pre_processor.calibrate(raw_data)

## Data Parsing
This section deals with segmenting the data into usable segements with correct labeling.

In [5]:
from src.data.data_helper import segement_data

In [6]:
segmented_data = segement_data(raw_data, window_length=200*5, overlap=50*5)

In [7]:
segmented_data.head()

,window_data,label,source
0,"[1.6104191404439578, 1.6104191404439578, 1.610...",0,C:\Users\erik\IES_codebase\EMG_Project\CDT406-...
1,"[1.6288106990610434, 1.6288106990610434, 1.628...",0,C:\Users\erik\IES_codebase\EMG_Project\CDT406-...
2,"[1.6324890107844603, 1.6324890107844603, 1.628...",0,C:\Users\erik\IES_codebase\EMG_Project\CDT406-...
3,"[1.6361673225078774, 1.6361673225078774, 1.636...",0,C:\Users\erik\IES_codebase\EMG_Project\CDT406-...
4,"[1.6251323873376262, 1.6251323873376262, 1.625...",0,C:\Users\erik\IES_codebase\EMG_Project\CDT406-...


In [8]:
segmented_data.iloc[50]

window_data    [1.4007553722091834, 1.411790307379435, 1.4265...
label                                                          2
source         C:\Users\erik\IES_codebase\EMG_Project\CDT406-...
Name: 50, dtype: object

In [9]:
num_classes = segmented_data['label'].nunique()
num_classes

4

In [10]:
y_data = np.array(segmented_data['label'].values)

# I apologize for this horrible line. Can be fixed by fixing the data_helper
X_data = np.stack(segmented_data.drop(columns=['label', 'source'])['window_data'].values)

In [11]:
X_data.shape

(9930, 1000)

In [12]:
X_data = pre_processor.batch_pre_process(X_data)
X_data.shape

(9930, 1000)

In [13]:
y_data.shape

(9930,)

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
        X_data, y_data, test_size=0.2
    )

In [15]:
X_train.shape

(7944, 1000)

In [16]:
X_val.shape

(1986, 1000)

## Model Setup

In [17]:
from src.models.LSTM.LSTM import LSTM
from src.models.LSTM_STFT.LSTM_STFT import LSTM_STFT
from src.models.LSTM_STFT_Dense.LSTM_STFT_Dense import LSTM_STFT_Dense

In [23]:
LSTM_model = LSTM(input_shape=X_data.shape[1], num_classes=num_classes)

In [24]:
LSTM_STFT_model = LSTM_STFT(input_shape=X_data.shape[1], num_classes=num_classes)

In [25]:
LSTM_STFT_Dense_model = LSTM_STFT_Dense(input_shape=X_data.shape[1], num_classes=num_classes)

# Train Model

In [26]:
LSTM_model.train(X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, epochs=10, verbose=2)

Epoch 1/10
249/249 - 3s - 12ms/step - accuracy: 0.8841 - loss: 0.4250 - val_accuracy: 0.9215 - val_loss: 0.2870
Epoch 2/10
249/249 - 1s - 4ms/step - accuracy: 0.9183 - loss: 0.3156 - val_accuracy: 0.9315 - val_loss: 0.2690
Epoch 3/10
249/249 - 1s - 5ms/step - accuracy: 0.9203 - loss: 0.3082 - val_accuracy: 0.9305 - val_loss: 0.2664
Epoch 4/10
249/249 - 1s - 5ms/step - accuracy: 0.9174 - loss: 0.3104 - val_accuracy: 0.9280 - val_loss: 0.2836
Epoch 5/10
249/249 - 1s - 4ms/step - accuracy: 0.9164 - loss: 0.3127 - val_accuracy: 0.9335 - val_loss: 0.2569
Epoch 6/10
249/249 - 1s - 4ms/step - accuracy: 0.9204 - loss: 0.3010 - val_accuracy: 0.9350 - val_loss: 0.2566
Epoch 7/10
249/249 - 1s - 5ms/step - accuracy: 0.9178 - loss: 0.3132 - val_accuracy: 0.9260 - val_loss: 0.2696
Epoch 8/10
249/249 - 1s - 4ms/step - accuracy: 0.9217 - loss: 0.2983 - val_accuracy: 0.9048 - val_loss: 0.3128
Epoch 9/10
249/249 - 1s - 4ms/step - accuracy: 0.9179 - loss: 0.3036 - val_accuracy: 0.9355 - val_loss: 0.2516


In [27]:
LSTM_STFT_model.train(X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, epochs=10, verbose=2)

Epoch 1/10
249/249 - 3s - 14ms/step - accuracy: 0.9110 - loss: 0.3652 - val_accuracy: 0.9048 - val_loss: 0.3204
Epoch 2/10
249/249 - 1s - 5ms/step - accuracy: 0.9213 - loss: 0.3048 - val_accuracy: 0.9295 - val_loss: 0.2600
Epoch 3/10
249/249 - 1s - 5ms/step - accuracy: 0.9233 - loss: 0.2981 - val_accuracy: 0.9355 - val_loss: 0.2606
Epoch 4/10
249/249 - 1s - 5ms/step - accuracy: 0.9230 - loss: 0.3013 - val_accuracy: 0.9315 - val_loss: 0.2541
Epoch 5/10
249/249 - 1s - 5ms/step - accuracy: 0.9222 - loss: 0.2978 - val_accuracy: 0.9325 - val_loss: 0.2598
Epoch 6/10
249/249 - 1s - 5ms/step - accuracy: 0.9223 - loss: 0.2967 - val_accuracy: 0.9290 - val_loss: 0.2698
Epoch 7/10
249/249 - 1s - 5ms/step - accuracy: 0.9248 - loss: 0.2905 - val_accuracy: 0.9325 - val_loss: 0.2549
Epoch 8/10
249/249 - 1s - 5ms/step - accuracy: 0.9270 - loss: 0.2864 - val_accuracy: 0.9325 - val_loss: 0.2521
Epoch 9/10
249/249 - 1s - 5ms/step - accuracy: 0.9265 - loss: 0.2868 - val_accuracy: 0.9330 - val_loss: 0.2604


In [28]:
LSTM_STFT_Dense_model.train(X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, epochs=10, verbose=2)

Epoch 1/10
249/249 - 3s - 12ms/step - accuracy: 0.8828 - loss: 0.4605 - val_accuracy: 0.9310 - val_loss: 0.2646
Epoch 2/10
249/249 - 1s - 5ms/step - accuracy: 0.9158 - loss: 0.3196 - val_accuracy: 0.9270 - val_loss: 0.2874
Epoch 3/10
249/249 - 1s - 4ms/step - accuracy: 0.9160 - loss: 0.3106 - val_accuracy: 0.9295 - val_loss: 0.2686
Epoch 4/10
249/249 - 1s - 4ms/step - accuracy: 0.9182 - loss: 0.3131 - val_accuracy: 0.9305 - val_loss: 0.2683
Epoch 5/10
249/249 - 1s - 4ms/step - accuracy: 0.9211 - loss: 0.3022 - val_accuracy: 0.9305 - val_loss: 0.2656
Epoch 6/10
249/249 - 1s - 4ms/step - accuracy: 0.9187 - loss: 0.3012 - val_accuracy: 0.9320 - val_loss: 0.2605
Epoch 7/10
249/249 - 1s - 5ms/step - accuracy: 0.9202 - loss: 0.3032 - val_accuracy: 0.9194 - val_loss: 0.2927
Epoch 8/10
249/249 - 1s - 5ms/step - accuracy: 0.9215 - loss: 0.3044 - val_accuracy: 0.9134 - val_loss: 0.2890
Epoch 9/10
249/249 - 1s - 4ms/step - accuracy: 0.9203 - loss: 0.3080 - val_accuracy: 0.9270 - val_loss: 0.2753
